# Imports

In [39]:
import numpy as np
from scipy.optimize import minimize
import random
import multiprocessing

import cvxpy as cp

In [2]:
np.random.seed(1)

# Create random Reachable sets

In [3]:
c = 25 # cache capacity
k = 5 # each user should be in at most K reachable sets
m = 2 # at most M items can be initially recommended to each user
number_of_items = 10
number_of_users = 12
items= np.array(range(0,number_of_items)) # videos
users = np.array(range(0,number_of_users)) # users
# initialize x0 for the optimization
x = np.full(len(items),0.5)
y = np.full((len(items), len(users)),0.5)
x0 = np.concatenate([x.flatten(), y.flatten()])
# Random Reachable sets
reachable_sets = np.array(np.array([[np.array(random.sample(list(users), random.choice(users))) for user in users] for i in items]))
I = np.zeros((len(items), len(users), len(users)))
for i in items:
    for u in users:
        for v in reachable_sets[i][u]:
            I[i][u][v] = 1

# Objective function

In [13]:
def objective(z):
    x = z[:len(items)]
    y = z[len(items):].reshape(len(items),len(users))
    
    #The first term in (1) counts the users to which an item is 
    #initially recommended and the users in the ensuing reachable sets
    #first_term = sum([(1+len(reachable_sets[i][u])) * y[i][u] for u in users])
    #The second term in (1) is due to the overlap of reachable sets Riu,Riv
    #second_term = 0.5*sum([sum([len(set(reachable_sets[i][u])&set(reachable_sets[i][v]))*y[i][u]*y[i][v] for v in users if v!=u]) for u in users])
    # Ekfonisi
    #return -sum([x[i]*(sum([(1+len(reachable_sets[i][u])) * y[i][u] for u in users]) - 0.5*sum([sum([len(set(reachable_sets[i][u])&set(reachable_sets[i][v]))*y[i][u]*y[i][v] for v in users if v!=u]) for u in users])) for i in items])
    # Paper
    return -sum([x[i]*sum([(1+len(reachable_sets[i][u])) * y[i][u] - 0.5*sum([len(set(reachable_sets[i][u])&set(reachable_sets[i][v]))*y[i][u]*y[i][v] for v in users if v!=u]) for u in users]) for i in items])

# Constraint (2) is the cache capacity constraint

In [4]:
def constraint2(z):  
    x = z[:len(items)]
    
    return sum([x[i] for i in items]) - c

# Constraint (3) says that at most M items can be initially recommended to each user

In [5]:
def constraint3(z):  
    y = z[len(items):].reshape(len(items),len(users))
    
    return [(-sum([y[i][u] for i in items]))+m for u in users]

# Constraint (4) says that each user should be in at most K reachable sets these capture content consumption, attention span or time constraints

In [6]:
def constraint4(z):  
    y = z[len(items):].reshape(len(items),len(users))
    
    return [-sum([sum([I[i][u][v]*y[i][u] for u in users]) for i in items])+k for u in users]

# Solve

In [14]:
con2 = {'type': 'eq', 'fun': constraint2} 
con3 = {'type': 'ineq', 'fun': constraint3} 
con4 = {'type': 'ineq', 'fun': constraint4} 
cons = ([con2, con3, con4])
# Bounds
b = (0.0,1.0)
bnds = [b for i in range(len(x0))]

# show initial objective
print('Initial Objective: ' + str(-1*objective(x0)))

solution = minimize(objective, x0, method='SLSQP', bounds=bnds, constraints=cons)

Initial Objective: 6.5


In [15]:
x = solution.x

# show final objective
print('Final Objective: ' + str(-1*objective(x)))

Final Objective: 139.25143468912935


In [16]:
solution

     fun: -139.25143468912935
     jac: array([-1.49977245e+01, -1.20000019e+01, -1.30000000e+01, -1.40005589e+01,
       -1.39978580e+01, -1.30047150e+01, -1.47501945e+01, -1.49999981e+01,
       -1.27501965e+01, -1.57501945e+01, -4.00000000e+00,  1.00000000e+00,
       -1.00000000e+00,  1.00000000e+00, -2.00000000e+00,  0.00000000e+00,
       -1.00000000e+00, -5.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        1.00000000e+00,  0.00000000e+00,  2.00000000e+00,  0.00000000e+00,
       -1.00000000e+00, -4.00000000e+00, -4.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  2.00000000e+00,  1.00000000e+00,
        0.00000000e+00, -1.00000000e+00,  4.99998665e+00,  9.99996185e-01,
       -1.00000000e+00,  1.99999046e+00,  1.99999237e+00, -3.81469727e-06,
        9.99996185e-01, -2.00000000e+00, -3.81469727e-06, -5.00001335e+00,
        2.99999237e+00, -1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        2.00000000e+00, -4.00000000e+00,  4.00000000e+00,  3

In [17]:
solution.x.shape

(130,)

In [18]:
x = solution.x[:len(items)].round()
y = solution.x[len(items):].reshape(len(items),len(users)).round()

# Solution

## Items in cache

In [19]:
x

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## Recommended items

In [20]:
y

array([[1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1.]])

In [47]:
y.shape

(10, 12)

In [ ]:
x = cp.Variable(x.shape)
y = cp.Variable(y.shape)
prob = cp.Problem(cp.Maximize(cp.sum(x*cp.sum())),
                 [x<=1,
                  x>=0,
                  y<=1,
                  y>=0
                     
#                   sum([x[i] for i in items]) == c,
#                  [sum([y[i][u] for i in items]) <= m for u in users],
#                  [sum([sum([I[i][u][v]*y[i][u] for u in users]) for i in items]) <= k for u in users]
                  ])
prob.solve()